In [1]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

In [23]:
"""Groups shortcuts
Cyclic groups definition C[x]."""
C = [CyclicPermutationGroup(1),CyclicPermutationGroup(1)]
for i in range(30):
     C.append(CyclicPermutationGroup(i+2))

"""Alternating group definition A[x]."""
A = [AlternatingGroup(4), AlternatingGroup(4), AlternatingGroup(4), AlternatingGroup(4)]
for i in range(15):
     A.append(AlternatingGroup(i+4))

"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(1,100):
    if i+1 not in [128,192]:
        small += gap.IdsOfAllSmallGroups(i+1)
# small += gap.IdsOfAllSmallGroups(96)

In [1]:
"""Function to return list of class sizes in default order."""
def class_sizes(G):
    group_order = G.order()
    reps = G.conjugacy_classes_representatives()
    class_sizes = []
    for g in reps:
        class_sizes.append(group_order / G.centralizer(g).order())
    return class_sizes

In [2]:
"""Function to find all subgroups. Returns list of subgroups, up to conjugacy."""
def subgroups(G):
    subgroups = G.conjugacy_classes_subgroups()
    return subgroups

In [3]:
"""Function to find induced trivial character of each subgroup. Returns a list of induced trivial characters of subgroups."""
def get_induced(subgroups, G):
    ind = []
    for subgroup in subgroups:
        ind.append(subgroup.trivial_character().induct(G))
    return ind

In [4]:
"""Forms the matrices that we use to solve the simultaneous equation later. 
matrix[1:-1]: Excludes the first and last subgroup (trivial subgroup and the group itself), and the first column.
checkmat[1:-1] Used for checking value at identity"""
def mat(induced):
    matrix = []
    checkmat = []
    for char in induced:
        matrix.append(list(char.values())[1:])
        checkmat.append(list(char.values())[0])
#    for line in matrix:
#        print(line)
    return matrix[1:-1], checkmat[1:-1]

In [5]:
"""Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise."""
def test(G):
    a = class_sizes(G)
    b = subgroups(G)
    c = get_induced(b, G)
    d, iden = mat(c)
    e = Matrix(d).T
    temp = []
    for i in range(len(a)-1):
        temp.append(1)
    Y = vector(temp)
    p = MixedIntegerLinearProgram(solver='GLPK')
    w = p.new_variable(nonnegative=True)
    p.add_constraint(e*w == Y)
    try:
        p.solve()
        out = []
        check = []
        for i, v in (p.get_values(w).items()):
            out.append('w_%s = %s' % (i, v))
            check.append(v)
            """Prints subgroup and subgroup order."""
            if round(v,6) != 0:
                print(b[i+1].order())
        """Check if identity is larger than others."""
        x = 0
        for i in range(len(iden)):
            x += int(iden[i])*float(check[i])
        """Run one of the two options below: Rational, Integer:
        Checks for Rational Solutions"""
        if x > 1:
            print(x)
            print(out)
            print(str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]))
        else:
            print("The value identity is equals to the rest:" + gap.IdSmallGroup(G))
        """Checks for Integer Solutions"""
#         if x > 1:
#             n = G.order()
#             mult = n/(x-1)
#             print(mult)
#             inte = True
#             for num in test:
#                 if round(mult*num - round(mult*num),6) > 0 or round(mult*num - round(mult*num),6) < 0:
#                     inte = False
#             if inte == True:
#                 print(gap.IdSmallGroup(G))
#         else:
#             print("The value identity is equals to the rest:" + gap.IdSmallGroup(G))
    except:
        print('fail')

In [6]:
"""Main function"""
if __name__ == "__main__":
    """If group is in id form.
    https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
    This works on CoCalc/non-windows system with gap_packages library installed."""
#     groups = small
#     for groupid in groups:
#         G = gap.SmallGroup(groupid)
#         #test(PermutationGroup(gap_group = G.AsPermGroup()))
#         try:
#             test(PermutationGroup(gap_group = G.AsPermGroup()))
#         except:
#             pass
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
    D1 = CyclicPermutationGroup(2)
    D2 = CyclicPermutationGroup(2)
    G = direct_product_permgroups([D1,D2])
    groups = [DihedralGroup(6)]
    for group in groups:
        try:
            test(group)
        except:
            pass
        
    """Extra: Count subgroups."""
#     D1 = CyclicPermutationGroup(3)
#     groups = [direct_product_permgroups([D1,D1,D1,D1])]
#     for group in groups:
#         a = group.subgroups()
#         count = 0
#         for group in a:
#             if group.order() == 27:
#                 count += 1
#         print(count)

2.0
[ 12, 4 ]
